# Classify Robot Framework Failures into Fix Categories

## Step 1: Load the structured failures JSON file

In [ ]:
import json
import pandas as pd

with open("structured_failures.json", "r") as f:
    data = json.load(f)

Add example fix categories manually based on error messages

You can later update or expand these categories

In [ ]:
for item in data:
    error = item["error"].lower()
    if "missing" in error and "argument" in error:
        item["fix_category"] = "missing_argument"
    elif "not found" in error or "selector" in error:
        item["fix_category"] = "invalid_selector"
    elif "assert" in error or "should be equal" in error:
        item["fix_category"] = "assertion_failed"
    elif "timeout" in error:
        item["fix_category"] = "timeout"
    elif "connection" in error:
        item["fix_category"] = "connection_error"
    else:
        item["fix_category"] = "other"

print(data)

Convert to DataFrame

In [ ]:
records = []
for item in data:
    msg = f"Test name: {item['test_name']}\n"
    msg += f"Doc: {item.get('doc', '')}\n"
    msg += f"Error: {item['error']}\n"
    for step in item.get("steps", []):
        msg += f"Step: {step['keyword']}\n"
        msg += f"Args: {' '.join(step['args'])}\n"
        msg += f"Status: {step['status']}\n"
        if step.get("doc"):
            msg += f"Doc: {step['doc']}\n"
        if step.get("messages"):
            msg += f"Messages: {' | '.join(step['messages'])}\n"
    records.append({
        "test_name": item["test_name"],
        "log_text": msg,
        "fix_category": item["fix_category"]
    })

#print(records)
print('\033[1m' + "Example log text :" + '\033[0m')
print(records[len(records) - 1])

df = pd.DataFrame(records)
df.head(len(df))

## Step 2: Embed the logs using TF-IDF

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, stop_words="english")
X = vectorizer.fit_transform([r["log_text"] for r in records])
y = [r["fix_category"] for r in records]

## Step 3: Train/test split + classifier training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

## Step 4: Evaluate

In [ ]:
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

## Step 5: Predict new failure

In [ ]:
new_data = {
    "test_name": "Connect without API key",
    "error": "TypeError: TestObject.__init__() missing 1 required positional argument: 'api_key'",
    "doc": "Attempts to connect to the server without providing API key.",
    "steps": [
        {
            "keyword": "Connect",
            "args": ["http://localhost"],
            "status": "FAIL",
            "depth": 0,
            "doc": "Connects to backend server using TestObject",
            "messages": ["Connecting to http://localhost", "Exception raised: missing 'api_key'"]
        }
    ]
}

log_text = f"Test name: {new_data['test_name']}\n"
log_text += f"Doc: {new_data['doc']}\n"
log_text += f"Error: {new_data['error']}\n"
for step in new_data["steps"]:
    log_text += f"Step: {step['keyword']}\n"
    log_text += f"Args: {' '.join(step['args'])}\n"
    log_text += f"Status: {step['status']}\n"
    if step.get("doc"):
        log_text += f"Doc: {step['doc']}\n"
    if step.get("messages"):
        log_text += f"Messages: {' | '.join(step['messages'])}\n"

print('\033[1m' + "New log text :\n" + '\033[0m' + log_text)

new_vec = vectorizer.transform([log_text])
pred = clf.predict(new_vec)
print("Prediction:", pred[0])

## Step 6: Similarity Retrieval with FAISS

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

# Reuse the same training set with structured log_texts
log_texts = [r["log_text"] for r in records]
metadata = [(r["test_name"], r["fix_category"]) for r in records]

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(log_texts, show_progress_bar=True, normalize_embeddings=True)

# Build FAISS index
index = faiss.IndexFlatIP(embeddings.shape[1])  # use cosine similarity with normalized vectors
index.add(np.array(embeddings))

# Encode query the same way
query_embedding = model.encode([log_text], normalize_embeddings=True)
D, I = index.search(query_embedding, k=3)

print("\nTop 3 similar past failures:")
for rank, idx in enumerate(I[0]):
    print(f"\n#{rank+1}")
    print("Test:", metadata[idx][0])
    print("Fix Category:", metadata[idx][1])
    print("Similarity Score:", D[0][rank])
    print("Log Snippet:\n", log_texts[idx][:400], "...")